In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Database connection and query
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

df_clients = pd.read_sql_query("SELECT * FROM core_client", conn)
df_sales = pd.read_sql("SELECT * FROM core_sale;", conn)
df_salesmans = pd.read_sql("SELECT * FROM core_salesman;", conn)
df_categories = pd.read_sql("SELECT * FROM core_category;", conn)
df_products = pd.read_sql("SELECT * FROM core_product;", conn)
df_productsales = pd.read_sql("SELECT * FROM core_productsale;", conn)
df_productsales

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn

,id,quantity,income,product_id,sale_id,income_currency
0,16921,4.10,14.35,17,00034279,USD
1,16953,4.00,14.00,17,00034290,USD
2,16930,8.10,43.52,22,00034282,USD
3,17005,6.80,21.87,23,00034311,USD
4,16906,24.00,46.72,33,00034269,USD
...,...,...,...,...,...,...
71828,18772,25.30,0.00,489,00034825,USD
71829,19432,4.30,0.00,489,00035023,USD
71830,19630,8.70,0.00,489,00035079,USD
71831,59878,4.40,0.00,489,00001021,USD


In [3]:
# Quantity of sales a client made with every salesman

df_merged = pd.merge(df_sales, df_salesmans, left_on='salesman_id', right_on='id', suffixes=('_sale', '_salesman'))
df_merged_clients = pd.merge(df_merged, df_clients, left_on='client_id', right_on='id', suffixes=('_sale', '_client'))
df_merged_clients.drop(columns=['id_sale','id_salesman','date', 'status', 'income_currency', 'description', 'identity_card_sale', 'image_sale', 'phone_1', 'phone_2', 'address_sale', 'identity_card_client', 'phone', 'address_client', 'image_client'], inplace=True)
df_merged_clients.rename(columns={'name_sale': 'name_salesman'}, inplace=True)
df_merged_clients.set_index(['client_id', 'salesman_id', 'name_salesman'], inplace=True)
df_merged_clients["count"] = df_merged_clients.groupby(['client_id', 'salesman_id'])['id'].count()
df_merged_clients["total_income"] = df_merged_clients.groupby(['client_id', 'salesman_id'])['income'].sum()
df_merged_clients.reset_index(inplace=True)
data = {
    "name_salesman": df_merged_clients.groupby(['client_id', 'salesman_id'])['name_salesman'].first(),
    "name_client": df_merged_clients.groupby(['client_id', 'salesman_id'])['name_client'].first(),
    "count": df_merged_clients.groupby(['client_id', 'salesman_id'])['count'].first(),
    'total_income': df_merged_clients.groupby(['client_id', 'salesman_id'])['total_income'].first()
}
df_aux = pd.DataFrame(data)
# df_aux.reset_index(inplace=True)
df_aux

name_salesman  \
client_id salesman_id                   
1         2                        V2   
2         1            VENTAS DIRECTA   
3         1            VENTAS DIRECTA   
4         2                        V2   
          4              Luis Guanipa   
...                               ...   
1072      1            VENTAS DIRECTA   
1073      2                        V2   
1074      1            VENTAS DIRECTA   
          7              Edgado Rojas   
1075      1            VENTAS DIRECTA   

                                                         name_client  count  \
client_id salesman_id                                                         
1         2                                 INVERSIONES CAREOCA, C.A      1   
2         1                                         VECINO DEFREITAS     16   
3         1                                                   KALIFA      1   
4         2            ARCOFRESCO PRODUCCION Y COMERCIALIZACION, C.A      1   
          4            ARCOFRESCO PRODUCCION Y COMERCIALIZACION, C.A      1   
...                                                              ...    ...   
1072      1                                               DOLIS DIAZ      1   
1073      2                                                  LA MEGA      1   
1074      1                                          JOAQUIN FEREIRA      1   
          7                                          JOAQUIN FEREIRA      1   
1075      1                    GUARDIA NACIONAL ALCABALA LOS MEDANOS      8   

                       total_income  
client_id salesman_id                
1         2                  105.33  
2         1                 6900.48  
3         1                   10.90  
4         2                   40.05  
          4                  146.02  
...                             ...  
1072      1                   28.20  
1073      2                  263.33  
1074      1                   52.67  
          7                   70.59  
1075      1                14316.14  

[1499 rows x 4 columns]

In [6]:
# Quantity and income for categories for a client
df_merged_sales = pd.merge(df_sales, df_productsales, left_on='id', right_on='sale_id', suffixes=('_sale', '_productsales'))
df_merged_with_products = pd.merge(df_merged_sales, df_products, left_on='product_id', right_on='id', suffixes=('_sales', '_products'))
df_merged_with_categories = pd.merge(df_merged_with_products, df_categories, left_on='category_id', right_on='id', suffixes=('_products', '_categories'))
df_final_merged = pd.merge(df_merged_with_categories, df_clients, left_on='client_id', right_on='id', suffixes=('_categories', '_client'))
df_final_merged.drop(columns=['date', 'description_sales', 'income_currency_sale', 'status', 
                    'price_1_currency', 'price_2_currency','price_3_currency',
                    'price_1', 'price_2', 'price_3',
                    'code', 'identity_card', 'image_client',
                    'address', 'phone', 'presentation',
                    'income_currency_productsales', 'cost',
                    'brand', 'cost_currency', 'description_products',
                    'image_categories', 'id', 'id_categories', 'salesman_id',
                    'income_sale', 'id_sale', 'id_products'
                    ], inplace=True)
df_final_merged.set_index(['client_id', 'category_id'], inplace=True)
df_final_merged['count'] = df_final_merged.groupby(['client_id', 'category_id'])['quantity'].sum()
df_final_merged['total_income'] = df_final_merged.groupby(['client_id', 'category_id'])['income_productsales'].sum()
df_final_merged.reset_index(inplace=True)
data = {
    "name_client": df_final_merged.groupby(['client_id', 'category_id'])['name'].first(),
    "name_category": df_final_merged.groupby(['client_id', 'category_id'])['name_categories'].first(),
    "client_id": df_final_merged.groupby(['client_id', 'category_id'])['client_id'].first(),
    "category_id": df_final_merged.groupby(['client_id', 'category_id'])['category_id'].first(),
    "count": df_final_merged.groupby(['client_id', 'category_id'])['count'].first(),
    "total_income": df_final_merged.groupby(['client_id', 'category_id'])['total_income'].first()
}

df_aux_2 = pd.DataFrame(data)
# df_aux_2.drop(columns=['category_id', 'client_id'], inplace=True)
# df_aux_2.reset_index(inplace=True)
df_aux_2

name_client  \
client_id category_id                                          
1         55                        INVERSIONES CAREOCA, C.A   
2         8                                 VECINO DEFREITAS   
          19                                VECINO DEFREITAS   
          21                                VECINO DEFREITAS   
          25                                VECINO DEFREITAS   
...                                                      ...   
1075      8            GUARDIA NACIONAL ALCABALA LOS MEDANOS   
          21           GUARDIA NACIONAL ALCABALA LOS MEDANOS   
          55           GUARDIA NACIONAL ALCABALA LOS MEDANOS   
          1056         GUARDIA NACIONAL ALCABALA LOS MEDANOS   
          1060         GUARDIA NACIONAL ALCABALA LOS MEDANOS   

                             name_category  client_id  category_id   count  \
client_id category_id                                                        
1         55                        HUEVOS          1           55    2.00   
2         8              CREMAS Y NATILLAS          2            8   12.00   
          19           BOLOGNAS Y FIAMBRES          2           19    4.83   
          21                    MORTADELAS          2           21    6.58   
          25              QUESOS MADURADOS          2           25    1.60   
...                                    ...        ...          ...     ...   
1075      8              CREMAS Y NATILLAS       1075            8   14.00   
          21                    MORTADELAS       1075           21    7.00   
          55                        HUEVOS       1075           55  271.00   
          1056              ADMINISTRACION       1075         1056   48.00   
          1060                 BEBIDAS UHT       1075         1060    2.00   

                       total_income  
client_id category_id                
1         55                 105.33  
2         8                   23.88  
          19                  18.87  
          21                  13.75  
          25                  20.31  
...                             ...  
1075      8                   25.82  
          21                  15.31  
          55               14272.67  
          1056                 0.00  
          1060                 2.35  

[7766 rows x 6 columns]

In [5]:
# Quantity and income for products for a client

df_merged_sales = pd.merge(df_sales, df_productsales, left_on='id', right_on='sale_id', suffixes=('_sale', '_productsales'))
df_merged_with_products = pd.merge(df_merged_sales, df_products, left_on='product_id', right_on='id', suffixes=('_sales', '_products'))
df_final_merged = pd.merge(df_merged_with_products, df_clients, left_on='client_id', right_on='id', suffixes=('_sales_products', '_client'))
df_final_merged.drop(columns=['date', 'description_sales', 'income_currency_sale', 'status', 
                    'price_1_currency', 'price_2_currency','price_3_currency',
                    'price_1', 'price_2', 'price_3',
                    'code', 'identity_card', 'image_client',
                    'address', 'phone', 'presentation',
                    'income_currency_productsales', 'cost',
                    'brand', 'cost_currency', 'description_products',
                    'salesman_id', 'income_sale', 'id_sale', 'image_sales_products',
                    'category_id', 'id_sales_products', 'id_productsales'
                    ], inplace=True)
df_final_merged.set_index(['client_id', 'product_id'], inplace=True)
df_final_merged['count'] = df_final_merged.groupby(['client_id', 'product_id'])['quantity'].sum()
df_final_merged['total_income'] = df_final_merged.groupby(['client_id', 'product_id'])['income_productsales'].sum()

# df_final_merged.reset_index(inplace=True)
df_final_merged

quantity  income_productsales   sale_id  \
client_id product_id                                            
315       489             24.0                 0.00  00034292   
          35              72.0               114.00  00034463   
          35              48.0                76.00  00034634   
          33              72.0               140.16  00034463   
          33              48.0                93.44  00034634   
...                        ...                  ...       ...   
177       115             24.0                25.44  NE010466   
          115             20.0                21.20  NE011457   
          118             48.0                50.88  NE010466   
999       115              6.0                 6.36  NE011497   
          221              6.0                 4.00  NE011497   

                                          name_sales_products  id_client  \
client_id product_id                                                       
315       489                                            NONE        315   
          35          CREMA DE LECHE 250 GRS X UNIDAD QUENACA        315   
          35          CREMA DE LECHE 250 GRS X UNIDAD QUENACA        315   
          33                  QUESO CREMA GABY 250GR X UNIDAD        315   
          33                  QUESO CREMA GABY 250GR X UNIDAD        315   
...                                                       ...        ...   
177       115                  YOGURT FIRRNE YOKA FRESA 150GR        177   
          115                  YOGURT FIRRNE YOKA FRESA 150GR        177   
          118                 YOGURT FIRME YOKA NATURAL 150GR        177   
999       115                  YOGURT FIRRNE YOKA FRESA 150GR        999   
          221                     GELATINA GELI FRESA 150 GRS        999   

                                                        name_client  count  \
client_id product_id                                                         
315       489         PANADERIA CHADAS C.A (PANADERIA CHADAS, C.A )   24.0   
          35          PANADERIA CHADAS C.A (PANADERIA CHADAS, C.A )  120.0   
          35          PANADERIA CHADAS C.A (PANADERIA CHADAS, C.A )  120.0   
          33          PANADERIA CHADAS C.A (PANADERIA CHADAS, C.A )  120.0   
          33          PANADERIA CHADAS C.A (PANADERIA CHADAS, C.A )  120.0   
...                                                             ...    ...   
177       115                                      CASA FARES, C.A.   44.0   
          115                                      CASA FARES, C.A.   44.0   
          118                                      CASA FARES, C.A.   48.0   
999       115            FUAN MOHAMAD KALED (LA ROOSELVET 2022) F.P    6.0   
          221            FUAN MOHAMAD KALED (LA ROOSELVET 2022) F.P    6.0   

                      total_income  
client_id product_id                
315       489                 0.00  
          35                190.00  
          35                190.00  
          33                233.60  
          33                233.60  
...                            ...  
177       115                46.64  
          115                46.64  
          118                50.88  
999       115                 6.36  
          221                 4.00  

[71833 rows x 8 columns]